# 参数管理

在选择了架构并设置了超参数后，我们就进入了训练阶段。
此时，我们的目标是*找到使损失函数最小化的模型参数值*。
经过训练后，我们将需要使用这些参数来做出未来的预测。
此外，有时我们希望提取参数，以便在其他环境中复用它们，
将模型保存下来，以便它可以在其他软件中执行，
或者为了获得科学的理解而进行检查。

之前的介绍中，我们只依靠深度学习框架来完成训练的工作，
而忽略了操作参数的具体细节。
本节，我们将介绍以下内容：

* 访问参数，用于调试、诊断和可视化；
* 参数初始化；
* 在不同模型组件间共享参数。

(**我们首先看一下具有单隐藏层的多层感知机。**)


In [12]:
import torch
from torch import nn
# net[0] = nn.Linear(4, 8),其中第一个全连接层包含（8，4）的权重参数和一个（8，）的权重系数
# net[1] = nn.ReLU()，
# net[2] = nn.Linear(8, 1)，其中第二个全连接层包含（1，8）的权重参数和一个（1，）的权重系数
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1)) 
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.1537],
        [-0.1687]], grad_fn=<AddmmBackward0>)

>`grad_fn=<AddmmBackward0>` 是 PyTorch **自动微分（Autograd）系统**的一部分，它表示**当前张量是由哪个反向传播函数（backward function）产生的**。

---

1. `grad_fn` 是什么？

- 每个通过**可微操作**（如线性变换、ReLU、加法等）创建的 `Tensor`，如果 `requires_grad=True` 或其输入需要梯度，PyTorch 会为其记录一个 `grad_fn`。
- `grad_fn` 是一个**函数对象**，用于在反向传播时计算梯度。
- 它构成了**动态计算图（computational graph）的节点**。

> ✅ 简单说：`grad_fn` 告诉 PyTorch：“当你需要计算我的梯度时，请调用这个函数。”

---

2. 为什么是 `<AddmmBackward0>`？

你的网络最后一层是：
```python
nn.Linear(8, 1)
```

而 `nn.Linear` 的底层实现本质上是：
```python
output = input @ weight.T + bias
```

在 PyTorch 中，这个操作被优化为一个原子操作：**`addmm`**（**add** + **m**atrix-**m**atrix multiplication）

- `addmm(beta, input, alpha, mat1, mat2)` ≈ `beta * input + alpha * (mat1 @ mat2)`
- 对于 `Linear`，通常简化为：`bias + input @ weight.T`

因此，最后一层的输出张量的 `grad_fn` 就是 **`AddmmBackward`** —— 表示“这个结果来自 `addmm` 操作，反向传播时要用 `AddmmBackward` 来求导”。

> 📌 `AddmmBackward0` 中的 `0` 只是 PyTorch 内部用于区分不同实例的编号，可忽略。

---

📚 常见的 `grad_fn` 类型举例

| 操作 | `grad_fn` |
|------|----------|
| `x + y` | `<AddBackward0>` |
| `x @ y` | `<MmBackward0>` |
| `torch.relu(x)` | `<ReluBackward0>` |
| `nn.Linear(...)` | `<AddmmBackward0>` |
| `loss.mean()` | `<MeanBackward1>` |



✅ 总结

| 问题 | 答案 |
|------|------|
| `grad_fn=<AddmmBackward0>` 是什么？ | 表示该张量由 `addmm` 操作（即 `bias + input @ weight.T`）产生，反向传播时将调用 `AddmmBackward` 函数计算梯度 |
| 为什么是 `Addmm`？ | 因为 `nn.Linear` 底层使用高效的 `addmm` 算子融合了矩阵乘法和偏置加法 |
| 为什么即使输入无梯度也有 `grad_fn`？ | 因为模型参数（weight/bias）需要梯度，PyTorch 仍需构建计算图以支持对参数的反向传播 |
| 能否去掉 `grad_fn`？ | 可以用 `with torch.no_grad():` 或对输出 `.detach()`，但会失去梯度追踪能力 |

> 💡 **核心理解**：  
> `grad_fn` 是 PyTorch **动态计算图的“回溯指针”**，它让 `loss.backward()` 知道如何一步步把梯度从 loss 传回每个参数。  
> `AddmmBackward0` 正是线性层在反向传播中的“身份标识”。

## [**参数访问**]

我们从已有模型中访问参数。
当通过`Sequential`类定义模型时，
我们可以通过索引来访问模型的任意层。
这就像模型是一个列表一样，每层的参数都在其属性中。
如下所示，我们可以检查第二个全连接层的参数。


In [13]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.0815, -0.0473, -0.1333, -0.0493,  0.2132, -0.1748, -0.0557,  0.2975]])), ('bias', tensor([-0.1306]))])


输出的结果告诉我们一些重要的事情：
首先，这个全连接层包含两个参数，分别是该层的权重和偏置。
两者都存储为单精度浮点数（float32）。
注意，参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此。

### [**目标参数**]

注意，每个参数都表示为***参数类的一个实例***。
要对参数执行任何操作，首先我们需要访问底层的数值。
有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。
下面的代码从第二个全连接层（即第三个神经网络层）提取偏置，
提取后返回的是一个参数类实例，并进一步访问该参数的值。


In [14]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1306], requires_grad=True)
tensor([-0.1306])


>参数是复合的对象，包含值、梯度和额外信息。

这就是我们需要显式参数值的原因。
除了值之外，我们还可以访问每个参数的梯度。
在上面这个网络中，由于我们还没有调用反向传播，所以参数的梯度处于初始状态。


In [15]:
net[2].weight.grad == None

True

### [**一次性访问所有参数**]

当我们需要对所有参数执行操作时，逐个访问它们可能会很麻烦。
当我们处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂，
因为我们需要递归整个树来提取每个子块的参数。
下面，我们将通过演示来比较访问第一个全连接层的参数和访问所有层。


In [16]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


这为我们提供了另一种访问网络参数的方式，如下所示。


In [17]:
net.state_dict()['2.bias'].data

tensor([-0.1306])

### [**从嵌套块收集参数**]

让我们看看，如果我们将多个块相互嵌套，参数命名约定是如何工作的。
我们首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。


In [18]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.5016],
        [-0.5016]], grad_fn=<AddmmBackward0>)

[**设计了网络后，我们看看它是如何工作的。**]


In [19]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。
下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。


In [20]:
rgnet[0][1][0].bias.data
#  (block 1): Sequential(
#      (0): Linear(in_features=4, out_features=8, bias=True) 这个偏置项

tensor([-0.2648, -0.4610,  0.2451, -0.3874,  0.2295,  0.2634, -0.3390, -0.1883])

## 参数初始化

知道了如何访问参数后，现在我们看看如何正确地初始化参数。
我们在 :numref:`sec_numerical_stability`中讨论了良好初始化的必要性。
深度学习框架提供默认随机初始化，
也允许我们创建自定义初始化方法，
满足我们通过其他规则实现初始化权重。


默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵，
这个范围是根据输入和输出维度计算出的。
PyTorch的`nn.init`模块提供了多种预置初始化方法。


### [**内置初始化**]

让我们首先调用内置的初始化器。
下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量，
且将偏置参数设置为0。


In [21]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0101,  0.0055, -0.0042,  0.0025]), tensor(0.))

我们还可以将所有参数初始化为给定的常数，比如初始化为1。


In [22]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

我们还可以[**对某些块应用不同的初始化方法**]。
例如，下面我们使用Xavier初始化方法初始化第一个神经网络层，
然后将第三个神经网络层初始化为常量值42。


In [23]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.3978, -0.4562,  0.4097,  0.4607])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### [**自定义初始化**]

有时，深度学习框架没有提供我们需要的初始化方法。
在下面的例子中，我们使用以下的分布为任意权重参数$w$定义初始化方法：

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


同样，我们实现了一个`my_init`函数来应用到`net`。


In [29]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0]) # m.named_parameters() 返回该模块所有可学习参数的 (name, tensor) 对
        nn.init.uniform_(m.weight, -10, 10) # 使用 均匀分布 初始化权重： w \sim U(-10, 10) 
        # m.weight.data.abs() 返回一个布尔张量，形状与 weight 相同
        #  元素为 True 当且仅当 |W_ij| ≥ 5
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2] # 第一层的权重的前两行

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 7.4478, -6.3797, -6.6372,  0.0000],
        [-0.0000, -6.4368, -9.5899,  6.4208]], grad_fn=<SliceBackward0>)

注意，我们始终可以直接设置参数。


In [30]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -5.3797, -5.6372,  1.0000])

## [**参数绑定**]

有时我们希望在多个层间共享参数：
我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。


In [31]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


这个例子表明第三个和第五个神经网络层的参数是绑定的。
它们不仅值相等，而且由相同的张量表示。
因此，如果我们改变其中一个参数，另一个参数也会改变。
这里有一个问题：当参数绑定时，梯度会发生什么情况？

答案是由于模型参数包含梯度，因此在反向传播期间第二个隐藏层
（即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的梯度会加在一起。


这句话描述的是 **参数绑定（parameter tying）或参数共享（parameter sharing）** 在反向传播时的**梯度累积行为**。我们来深入解释。

```python
shared = nn.Linear(8, 8)   # 同一个对象
net = nn.Sequential(
    nn.Linear(4, 8),        # net[0]
    nn.ReLU(),              # net[1]
    shared,                 # net[2] ← 第一次使用 shared
    nn.ReLU(),
    shared,                 # net[4] ← 第二次使用 shared（同一个对象！）
    nn.ReLU(),
    nn.Linear(8, 1)
)
```

- `net[2]` 和 `net[4]` 是**完全相同的 `nn.Linear` 对象**
- 它们的 `weight` 和 `bias` 是**同一块内存**

1. 🔁 前向传播（Forward Pass）

输入 `X` 经过：
```
X → ... → H1 → [shared] → H2 → [shared] → H3 → ...
```

- 第一次调用 `shared(H1)` → 输出 `H2`
- 第二次调用 `shared(H2)` → 输出 `H3`

计算图中，`shared.weight` 被**用了两次**。

2. 📉 反向传播（Backward Pass）：关键来了！

假设最终损失为 `L`。

根据链式法则，`shared.weight` 的总梯度是：

$$
\frac{\partial L}{\partial W} = 
\underbrace{\frac{\partial L}{\partial H3} \cdot \frac{\partial H3}{\partial W}}_{\text{来自第二次使用（net[4]）}} +
\underbrace{\frac{\partial L}{\partial H2} \cdot \frac{\partial H2}{\partial W}}_{\text{来自第一次使用（net[2]）}}
$$

> ✅ **因为同一个参数 `W` 出现在计算图的两个地方，它的梯度是两路梯度的和！**

3. 🧠 为什么说“梯度会加在一起”？

- 每次参数被用于前向计算，Autograd 就会记录一条“路径”
- 反向传播时，**所有路径的梯度都会回传到该参数**
- PyTorch 默认对重复使用的参数执行 **梯度累加（gradient accumulation）**

> 这就像两个人同时拉一根绳子 —— 最终的力是两人之力的合力。

⚠️ 注意：这不是 bug，而是正确行为！

- 如果不加在一起，就违反了链式法则
- 梯度累加确保了**参数更新方向是全局最优的**
- 这正是参数共享能有效训练的数学基础

---

✅ 举个简单例子

假设 `W` 是标量，网络为：
```python
y = W * x1 + W * x2 = W(x1 + x2)
```
则：
$$
\frac{\partial y}{\partial W} = x1 + x2
$$

这正是两路梯度（`x1` 和 `x2`）相加的结果！

在你的网络中，只是把标量推广到了矩阵，把直接乘法推广到了多层非线性变换，但**原理完全相同**。

📌 总结

| 说法 | 解释 |
|------|------|
| “第二个隐藏层（net[2]）和第三个隐藏层（net[4]）的梯度会加在一起” | 因为它们是同一个参数对象，在反向传播时，来自两个不同位置的梯度信号会**累加**到同一个 `.grad` 张量中 |
| 这是否正确？ | ✅ 完全正确！这是自动微分的自然结果 |
| 对训练有何影响？ | 参数更新会同时考虑它在**所有使用位置**的影响，使共享参数能平衡多方需求 |

> 💡 **核心理解**：  
> **参数共享 ≠ 梯度只算一次，而是梯度从多个路径汇聚求和。**  
> 这正是共享参数能够学习到“通用表示”的关键机制。

## 小结

* 我们有几种方法可以访问、初始化和绑定模型参数。
* 我们可以使用自定义初始化方法。

## 练习

1. 使用 :numref:`sec_model_construction` 中定义的`FancyMLP`模型，访问各个层的参数。
1. 查看初始化模块文档以了解不同的初始化方法。
1. 构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。
1. 为什么共享参数是个好主意？


[Discussions](https://discuss.d2l.ai/t/1829)


为什么共享参数是个好主意？

---
这是一个非常深刻的问题！你展示的代码**正确实现了参数共享（parameter sharing）**，而“为什么这是个好主意”涉及深度学习中多个核心思想。
下面我们从**原理、优势、应用场景**三个层面详细解释。

---
🌟 为什么参数共享是个好主意？

1. **大幅减少模型参数量 → 防止过拟合**
- 假设不用共享：两个 `Linear(8,8)` 需要 `2 × (8×8 + 8) = 144` 个参数
- 使用共享：只需 `1 × (8×8 + 8) = 72` 个参数
- **参数减半！** 模型更简单，在小数据集上更不容易过拟合

> 💡 **奥卡姆剃刀原则**：能用更少参数解决问题，就不要用更多。

2. **强制模型学习通用特征表示（inductive bias）**
- 共享层必须同时适应**不同位置/阶段**的输入
- 这迫使它学习**更鲁棒、更通用的特征变换**
- 类似于“一个工具干多份活”，必须足够灵活

> 📌 这是一种**归纳偏置（inductive bias）** —— 我们告诉模型：“相信同一套变换在不同地方都有效”。

3. **实现“递归”或“深度展开”结构**
你的网络结构实际上是：
```
x → Linear → ReLU → [Shared] → ReLU → [Shared] → ReLU → Linear
```
这相当于对输入**反复应用同一个变换两次**。

这类似于：
- **RNN / Transformer 中的权重共享**（同一套参数处理所有时间步）
- **深度均衡模型（Deep Equilibrium Models）**
- **神经微分方程（Neural ODEs）** 的离散近似

> ✅ 共享参数让模型具备**迭代 refinement** 的能力。

4. **提高样本效率（data efficiency）**
- 每次前向传播，共享层被**多次更新**（通过不同路径的梯度）
- 相当于“一个参数接受多份训练信号”
- 在数据有限时，能更快学到有用表示

5. **经典成功案例：卷积神经网络（CNN）**
- CNN 的核心就是**参数共享**！
- 同一个卷积核（filter）在图像**所有位置滑动**
- 如果不共享，每个位置都要独立参数 → 参数爆炸，且无法泛化到新位置

> 🎯 **没有参数共享，就没有现代 CNN！**

6. **其他重要应用**
| 模型/技术 | 如何使用参数共享 |
|---------|----------------|
| **Siamese Networks** | 两个分支共享权重，用于度量学习（如人脸识别） |
| **Autoencoders** | 编码器和解码器有时权值绑定（tied weights） |
| **Transformers** | 多头注意力中，不同 head 有时共享投影矩阵 |
| **Meta-Learning** | MAML 等算法依赖共享结构快速适应新任务 |


⚠️ 注意：参数共享也有代价

- **约束了模型表达能力**：如果两个位置真的需要不同变换，共享会 hurt 性能
- **设计需谨慎**：不是所有层都适合共享（比如输入/输出层通常不共享）

但**在合适场景下，收益远大于代价**。

---

✅ 总结：为什么参数共享是个好主意？

| 优势 | 说明 |
|------|------|
| **减少参数** | 降低过拟合风险，节省内存和计算 |
| **引入归纳偏置** | 强制学习通用、可复用的特征变换 |
| **支持递归结构** | 实现深度迭代、时间序列建模等 |
| **提升数据效率** | 单个参数接收多路梯度信号 |
| **是许多SOTA模型的基础** | CNN、RNN、Transformer 等都依赖它 |

> 💡 **核心思想**：  
> **“如果一个问题在不同地方重复出现，就用同一个解决方案去处理它。”**  
> 这不仅是工程优化，更是对数据内在结构的一种假设。
